In [20]:
import utils as u
import importlib
importlib.reload(u)
import io
import requests
import os
import pandas as pd
from collections import Counter
from IPython.display import display, Markdown, Latex


from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import *  # noqa: F403
from pyspark.sql import functions as F  # noqa: F401
from pyspark.sql import SparkSession  # noqa: F401


from astroquery.imcce import Skybot
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u

In [21]:
r_orb = requests.post(
  'https://fink-portal.org/api/v1/ssocand',
  json={
    'kind': 'orbParams', # Mandatory, `orbParams` or `lightcurves`
  }
)
r_tra = requests.post(
  'https://fink-portal.org/api/v1/ssocand',
  json={
    'kind': 'lightcurves', # Mandatory, `orbParams` or `lightcurves`
  }
)

# Format output in a DataFrame
candid_reconstr_orbit = pd.read_json(io.BytesIO(r_orb.content))
candid_reconstr_traje = pd.read_json(io.BytesIO(r_tra.content))

In [22]:
def skybot_crossmatch(ra, dec, time):
    coord = SkyCoord(ra, dec, unit=u.degree)
    epoch = Time(time, format="jd")
    results = Skybot.cone_search(coord, 5*u.arcsec, epoch).to_pandas()
    return results[["Name", "centerdist"]]

In [23]:
skybot_schema = StructType(
        [
            StructField("Name", StringType(), True),
            StructField("centerdist", FloatType(), True)
        ]
    )

In [40]:
@pandas_udf(skybot_schema)
def skybot_crossmatch(ra, dec, jd):
    
    res_skybot = []
    
    for ra, dec, jd in zip(ra.values, dec.values, jd.values):
        try:
            coord = SkyCoord(ra, dec, unit=u.degree)
            epoch = Time(jd, format="jd")
            results = Skybot.cone_search(coord, 5*u.arcsec, epoch).to_pandas()[["Name", "centerdist"]]
            res_skybot.append(results)
        except RuntimeError:
            res_skybot.append(pd.DataFrame({"Name": ["null"], "centerdist": [-1.0]}))
            
    return pd.concat(res_skybot)

In [48]:
df_traj = spark.createDataFrame(candid_reconstr_traje).repartition(3000)

In [49]:
df_traj = df_traj.withColumn("skybot_res", skybot_crossmatch(df_traj["d:ra"], df_traj["d:dec"], df_traj["d:jd"]))

In [50]:
df_skybot_local = df_traj.toPandas()

In [58]:
df_skybot_local["Ast_Name"] = df_skybot_local["skybot_res"].str[0]
df_skybot_local["centerdist"] = df_skybot_local["skybot_res"].str[1]

In [60]:
df_skybot_local.drop("skybot_res", axis=1).to_parquet("skybot_result.parquet")